In [1]:
import torch.nn as nn

from model import LSTMModel
net = LSTMModel(6,0,12,0.1, 9, 9, 1, 6, 2)
net.load('models/lstm_net_1000.pkl')
print(net)

TypeError: 'int' object has no attribute '__getitem__'